In [ ]:
import tensorflow as tf
import numpy as np
import cifar10_input
import cifar10_model
import matplotlib.pyplot as plt
import os
from datetime import datetime
# 验证
def evaluation():
    with tf.Graph().as_default():
#         print(BATCH_SIZE)
        n_test = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL
        eval_images, eval_lables = cifar10_input.inputs('/root/code/cifar_10/cifar10-CNN-master/cifar10_data/cifar-10-batches-bin',128)
        print('这是eval_images,eval_lables ',eval_images,eval_lables)
        eval_logits = cifar10_model.inference(eval_images)
        print('构建完网络',eval_logits)
        result = tf.nn.softmax(eval_logits)
        print("这是预测结果矩阵",result)
        # tf.nn.in_top_k(predictions, targets, k, name=None)
        # 每个样本的预测结果的前k个最大的数里面是否包括包含targets预测中的标签，一般取1，
        # 即取预测最大概率的索引与标签的对比
        top_k_op = tf.nn.in_top_k(eval_logits, eval_lables, 1)
        print('这是top_k_op  ',top_k_op)
        saver = tf.train.Saver()
        with tf.Session() as session:
            ckpt = tf.train.get_checkpoint_state('signal_GPU/saver')
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(session, ckpt.model_checkpoint_path)
            
            predints1=session.run(eval_logits)
            print('这是预测1',predints1)
            predints2=session.run(result)
            print('这是预测2',predints2)


            
#             result.append(str(predints[0]))
            
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=session, coord=coord)
            num_iter = int(n_test / BATCH_SIZE)
            true_count = 0
            for step in range(num_iter):
                predictions = session.run(top_k_op)
#                 print('这是predictions',predictions)
                true_count = true_count + np.sum(predictions)
            precision = true_count / (num_iter * BATCH_SIZE)
            print('precision=', precision)
            coord.request_stop()
            coord.join(threads)
evaluation()    


/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.

Instructions for updating:

In [ ]:
def predict10(sess, adds):
    import json
    num_imgs = len(adds)
    result = []
    for i in range(num_imgs):
        data = json.loads(adds[i])
        predints=sess.run('y_conv:0',feed_dict={"x:0":data,"keep_prob:0": 1.0})
        result.append(str(predints[0]))
    
    return result